In [45]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg._interface import _CustomLinearOperator

import jlinops
from jlinops import DiagonalizedOperator, Gaussian1DBlurOperator

In [46]:
N = 10
blur_op = Gaussian1DBlurOperator(N, 3.0, mode="wrap")

In [62]:
def convert_to_diagonalized(linear_operator, how="C"):
    """Attemps to build a DiagonalizedOperator.

    C: circulant (basis is 1D FFT)
    BCCB: block-circulant-circulant-blocks (basis is 2D FFT)
    """
    valid_hows = ["C"]
    assert how in valid_hows, f"Invalid choice of how, must be one of {valid_hows}"
    

    # Create P operator
    in_shape = (linear_operator.shape[1],)
    if how == "C":
        
        def _matvec(x):
            return np.fft.ifft( x, norm="ortho" )
        
        def _rmatvec(x):
            return np.real(np.fft.fft( x, norm="ortho"))
        
        P = LinearOperator( linear_operator.shape, _matvec, _rmatvec  )

    else:
        raise NotImplementedError
    
    # Draw random vector
    rand_vec = np.clip( 10.0 + np.random.normal(size=linear_operator.shape[1]) , a_min = 1, a_max = 20 )
    tmp = P.matvec(rand_vec)
    tmp = linear_operator.matvec(tmp)
    tmp = P.rmatvec(rand_vec)
    eigenvalues = tmp/rand_vec

    return DiagonalizedOperator(P, eigenvalues)

In [63]:
P = convert_to_diagonalized(blur_op, how="C")

In [65]:
z = np.random.normal(size=10)
np.linalg.norm( (P @ z) - (blur_op @ z)  )

2.6953014165799387

In [17]:
np.fft.rfft(np.ones(5))

array([5.+0.j, 0.+0.j, 0.+0.j])

In [43]:
z = np.ones(11)
rfft = np.fft.rfftn(z, norm="ortho")
fft = np.fft.fftn(z, norm="ortho")

In [44]:
print(rfft)
print(fft)

[ 3.31662479e+00+0.j -6.69489674e-17+0.j  0.00000000e+00+0.j
 -6.69489674e-17+0.j  0.00000000e+00+0.j  1.67372418e-17+0.j]
[3.31662479e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
 0.00000000e+00+0.j 0.00000000e+00+0.j 1.67372418e-17+0.j
 1.67372418e-17+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
 0.00000000e+00+0.j 0.00000000e+00+0.j]


In [40]:
def rfft_to_fft_shape(rfft_vec):
    """Takes output of rfft and reshapes to size of output from fft."""
    return None

In [41]:
print(rfft.shape)

(6,)


In [42]:
print(fft.shape)

(11,)


In [10]:
rand_vec = np.clip( 10.0 + np.random.normal(size=N) , a_min = 1, a_max = 20 )
matvec_rand_vec = np.fft.ifftn(blur_op.matvec(np.fft.fftn(rand_vec, norm="ortho")), norm="ortho")
diagonalization = matvec_rand_vec/rand_vec

In [11]:
diagonalization

array([ 1.00000000e+00+6.78008234e-19j,  1.69241383e-01-7.77383709e-20j,
        8.35542922e-04+2.99010953e-19j, -1.85021865e-05-6.87684465e-20j,
       -3.71641180e-06-1.31116701e-19j,  1.81225122e-05-7.75158179e-20j,
       -3.71641180e-06-1.50798663e-19j, -1.85021865e-05-7.00528315e-20j,
        8.35542922e-04+4.23174617e-19j,  1.69241383e-01-7.82739581e-20j])

In [8]:
rand_vec = np.clip( 10.0 + np.random.normal(size=N) , a_min = 1, a_max = 20 )
matvec_rand_vec = np.fft.ifftn(blur_op.matvec(np.fft.ifft(rand_vec, norm="ortho")), norm="ortho")
diagonalization2 = matvec_rand_vec/rand_vec

In [9]:
diagonalization2

array([ 1.00000000e+00+7.84856427e-19j,  1.44117612e-01+1.32744908e-19j,
        8.13590536e-04+3.77504910e-19j, -1.97935429e-05+1.37100034e-19j,
       -4.00663558e-06-3.21796343e-19j,  1.81225122e-05+1.31579191e-19j,
       -3.44721060e-06-2.98486773e-19j, -1.72950798e-05+1.28155449e-19j,
        8.58087630e-04+3.87690787e-19j,  1.98744937e-01+1.55886096e-19j])

In [ ]:

def build_diagonalizable_operator_from_operator(
    linear_operator: LinearOperator,
    how: str = "bccb",
    with_cupy = False,
    trust=False,
    ):
    """
    Given a matrix that is diagonalizable, computes the diagonalization
    and returns a operator that performs matvecs using this basis.

    :param mat: a :py:class:`MatrixOperator` representing the matrix.
    :param how: how the operator is diagonalizable. Currently only BCCB is supported.
    """
   
    
    if with_cupy:
        assert CUPY_INSTALLED, "Unavailable, CuPy not installed."
        xp = cp
    else:
        xp = np

    # Make sure the method is supported
    if how == "bccb":

        # Compute the diagonalization
        input_shape = linear_operator.input_shape
        rand_vec = xp.clip( 10.0 + xp.random.normal(size=input_shape) , a_min = 1, a_max = 20 )
        matvec_rand_vec = xp.fft.fftn(linear_operator.matvec_shaped(xp.fft.ifftn(rand_vec, norm="ortho")), norm="ortho")
        diagonalization = matvec_rand_vec/rand_vec

        # Do it again, and check we get same result
        rand_vec2 = xp.clip( 10.0 + xp.random.normal(size=input_shape) , a_min = 1, a_max = 20 )
        matvec_rand_vec2 = xp.fft.fftn(linear_operator.matvec_shaped(xp.fft.ifftn(rand_vec2, norm="ortho")), norm="ortho")
        diagonalization2 = matvec_rand_vec2/rand_vec2

        if not trust:
            assert xp.linalg.norm(diagonalization - diagonalization2, ord=2)/math.prod(input_shape) < 1e-6, "Matrix operator doesn't appear to be diagonalized by an FFT."

        return BCCBOperator(diagonalization)

    else:
        raise NotImplementedError("Not currently supported.")

